In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchvision
!pip install yolov5

In [ ]:
import torch
from PIL import Image
import cv2
import json
import os
from sklearn.metrics import accuracy_score
from pycocotools.coco import COCO
# 加载预训练的 YOLOv5x 模型
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

coco = COCO('/content/drive/MyDrive/instances_train2017.json')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-11 Python-3.10.12 torch-2.2.1+cu121 CPU

100%|██████████| 166M/166M [00:01<00:00, 99.0MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


loading annotations into memory...
Done (t=22.65s)
creating index...
index created!


In [ ]:
# 获取所有类别
categories = coco.loadCats(coco.getCatIds())

# 打印类别名称
for category in categories:
    print(category['id'], category['name'])


1 person
2 bicycle
3 car
4 motorcycle
5 airplane
6 bus
7 train
8 truck
9 boat
10 traffic light
11 fire hydrant
13 stop sign
14 parking meter
15 bench
16 bird
17 cat
18 dog
19 horse
20 sheep
21 cow
22 elephant
23 bear
24 zebra
25 giraffe
27 backpack
28 umbrella
31 handbag
32 tie
33 suitcase
34 frisbee
35 skis
36 snowboard
37 sports ball
38 kite
39 baseball bat
40 baseball glove
41 skateboard
42 surfboard
43 tennis racket
44 bottle
46 wine glass
47 cup
48 fork
49 knife
50 spoon
51 bowl
52 banana
53 apple
54 sandwich
55 orange
56 broccoli
57 carrot
58 hot dog
59 pizza
60 donut
61 cake
62 chair
63 couch
64 potted plant
65 bed
67 dining table
70 toilet
72 tv
73 laptop
74 mouse
75 remote
76 keyboard
77 cell phone
78 microwave
79 oven
80 toaster
81 sink
82 refrigerator
84 book
85 clock
86 vase
87 scissors
88 teddy bear
89 hair drier
90 toothbrush


In [ ]:
import urllib.request

# 假设我们只对“船”类别感兴趣
interested_categories = ['boat']

# 获取这些类别的ID
catIds = coco.getCatIds(catNms=interested_categories)

# 分别获取每个类别的图像ID
imgIds = []
for catId in catIds:
    imgIds.extend(coco.getImgIds(catIds=[catId]))

# 移除重复的图像ID
imgIds = list(set(imgIds))

# 创建一个目录来保存下载的图像
os.makedirs('/content/drive/MyDrive/boatimages', exist_ok=True)

# 下载图像
for imgId in imgIds:
    img_info = coco.loadImgs(imgId)[0]
    img_url = img_info['coco_url']
    img_filename = os.path.join('/content/drive/MyDrive/boatimages', img_info['file_name'])
    if not os.path.exists(img_filename):
        urllib.request.urlretrieve(img_url, img_filename)


In [ ]:
# 预测并保存结果
predictions = []

for imgId in imgIds:
    img_info = coco.loadImgs(imgId)[0]
    img_filename = os.path.join('/content/drive/MyDrive/boatimages', img_info['file_name'])

    if os.path.exists(img_filename):
        image = Image.open(img_filename).convert('RGB')

        # 预测结果
        results = model(image)
        results_data = results.pandas().xyxy[0]  # 以pandas DataFrame格式获取结果

        # 仅保留boat类别的预测结果
        boat_results = results_data[results_data['name'] == 'boat']

        for _, row in boat_results.iterrows():
            predictions.append({
                'image_id': imgId,
                'category_id': catIds[0],  # 这里仅有boat类别
                'bbox': [row['xmin'], row['ymin'], row['xmax'] - row['xmin'], row['ymax'] - row['ymin']],
                'score': row['confidence']
            })


In [ ]:
# 保存预测结果为JSON文件
with open('/content/drive/MyDrive/boat.json', 'w') as f:
    json.dump(predictions, f)


In [ ]:
from pycocotools.cocoeval import COCOeval

# 加载预测结果和真值
coco_gt = COCO('/content/drive/MyDrive/instances_train2017.json')
coco_dt = coco_gt.loadRes('/content/drive/MyDrive/boat.json')

# 初始化COCOeval对象
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')

# 选择要评估的类别
coco_eval.params.catIds = catIds

# 运行COCOeval
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

loading annotations into memory...
Done (t=37.89s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.19s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=13.59s).
Accumulating evaluation results...
DONE (t=0.85s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.406
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.637
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.423
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.253
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.504
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.647
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.187
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.452
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe